In [1]:
#IMPORTING NECESSARY PACKAGES
import time
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_ta as pa
import yfinance as yf
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate, TimeDistributed
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
#HERE WE SET SETTINGS
iterations = 30
starting_time = '2000-01-01'
inputnumber = 26
today = datetime.now()
tomorrow = today + timedelta(days=1)
tomorrow_str = tomorrow.strftime('%Y-%m-%d')
tomorrow_str
ending_time = tomorrow_str
tomorrow_str
data = yf.download(tickers = '^GSPC', start = starting_time, end = ending_time)
start_time = time.time()
testingdata = []

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['^GSPC']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2000-01-01 -> 2025-03-24)')


In [3]:
#HERE WE FETCH RELEVANT MARKET DATA
#tech = yf.download(tickers = 'XLK', start = starting_time, end = ending_time)
#tech = tech[['Open', 'Adj Close', 'High', 'Low']]
#tech = tech.rename(columns={'Open': 'techOpen', 'Adj Close': 'techAdj Close', 'High': 'techHigh', 'Low': 'techLow'})
#display(tech)
#data = pd.concat([data, tech], axis=1)
#ftse = yf.download(tickers = '^FTSE', start = starting_time, end = ending_time)
#ftse = ftse[['Open', 'Adj Close', 'High', 'Low']]
#ftse = ftse.dropna()

#ftse = ftse.rename(columns={'Open': 'FTSEOpen', 'Adj Close': 'FTSEAdj Close', 'High': 'FTSEHigh', 'Low': 'FTSELow'})
#data = pd.concat([data, ftse], axis=1)
#gc = yf.download(tickers = 'GCSIX', start = starting_time, end = ending_time)
#gc = gc[['Open', 'Adj Close', 'High', 'Low']]
#gc = gc.rename(columns={'Open': 'GCOpen', 'Adj Close': 'GCAdj Close', 'High': 'GCHigh', 'Low': 'GCLow'})
#data = pd.concat([data, gc], axis=1)
#bond = yf.download(tickers = 'FBNDX', start = starting_time, end = ending_time)#bond = bond[['Open', 'Adj Close']]
#bond = bond.rename(columns={'Open': 'FBOpen', 'Adj Close': 'FBAdj Close', 'High': 'FBHigh', 'Low': 'FBLow'})
#data = pd.concat([data, bond], axis=1)
#bond = yf.download(tickers = 'TLT', start = starting_time, end = ending_time)
#bond = bond[['Open', 'Adj Close']]
#bond = bond.rename(columns={'Open': 'BOpen', 'Adj Close': 'BAdj Close'})
#data = pd.concat([data, bond], axis=1)
#nk = yf.download(tickers = 'NKD=F', start = starting_time, end = ending_time)
#nk = nk[['Open', 'Adj Close']]
#nk = nk.rename(columns={'Open': 'NKOpen', 'Adj Close': 'NKClose',})
#data = pd.concat([data, nk], axis=1)
volatility = yf.download(tickers = '^VIX', start = starting_time, end = ending_time)
volatility = volatility[['Open', 'Adj Close', 'High', 'Low']]
volatility = volatility.rename(columns={'Open': 'VOpen', 'Adj Close': 'VAdj Close', 'High': 'VHigh', 'Low': 'VLow'})
data = pd.concat([data, volatility], axis=1)
DMP = pa.dm(data['High'], data['Low'], length=16)
data = data.join(DMP)
macd_df = ta.macd(data['Adj Close'])
data['MACD'] = macd_df['MACDs_12_26_9']
#data.ta.pivot(levels=[1])
USD = yf.download(tickers = '^NYICDX', start = starting_time, end = ending_time)
USD = USD[['Adj Close', 'Open', 'High', 'Low']]
USD = USD.rename(columns={'Adj Close': 'USDAdj Close', 'Open': 'USDOpen', 'High': 'USDHigh', 'Low': 'USDLow'})
#display(USD)
USD = USD.dropna()
USD.tail(20)
data = pd.concat([data, USD], axis=1)
#data['Pivot Point'] = (data['High'] + data['Low'] + data['Close']) / 3
#data['R1'] = (2 * data['Pivot Point']) - data['Low']
#data['S1'] = (2 * data['Pivot Point']) - data['High']
#data['R2'] = data['Pivot Point'] + (data['High'] - data['Low'])
#data['S2'] = data['Pivot Point'] - (data['High'] - data['Low'])
#data['FTSEATR'] = ta.atr(data['FTSEHigh'], data['FTSELow'], data['FTSEAdj Close'], length=16)
#data['FTSERSI'] = ta.rsi(data['FTSEAdj Close'], length=15)
#data['FBRSI'] = ta.rsi(data['FBAdj Close'], length=15)
#data['FBATR'] = ta.atr(data['FBHigh'], data['FBLow'], data['FBAdj Close'], length=16)
#data['techRSI'] = ta.rsi(data['techAdj Close'], length=15)
#data['techATR'] = ta.atr(data['techHigh'], data['techLow'], data['techAdj Close'], length=16)
#data['GCRSI'] = ta.rsi(data['GCAdj Close'], length=15)
#data['GCATR'] = ta.atr(data['GCHigh'], data['GCLow'], data['GCAdj Close'], length=16)
#data['BRSI'] = ta.rsi(data['BAdj Close'], length=15)
data['USDRSI'] = ta.rsi(data['USDAdj Close'], length=15)
data['USDATR'] = ta.atr(data['USDHigh'], data['USDLow'], data['USDAdj Close'], length=16)
#data['VRSI'] = ta.rsi(data['VAdj Close'], length=15)
#data['VATR'] = ta.atr(data['VHigh'], data['VLow'], data['VAdj Close'], length=16)
data['RSI'] = ta.rsi(data['Adj Close'], length=15)
data['ATR'] = ta.atr(data['High'], data['Low'], data['Adj Close'], length=16)
data['EMAF'] = ta.ema(data['Adj Close'], length=20)
data['EMAM'] = ta.ema(data['Adj Close'], length=100)
data['EMAS'] = ta.ema(data['Adj Close'], length=150)


data['Target'] = data['Adj Close'] - data['Open']
data['Target'] = data['Target'].shift(-1)
data['TargetClass'] = [1 if data.Target[i]>0 else 0 for i in range(len(data))]
data['TargetNextClose'] = data['Adj Close'].shift(-1)
data.dropna()


#display(data)
#data.dropna(inplace=True)
data = data.iloc[150:]
#data.fillna(0, inplace=True)
#data = data.drop(['Volume', 'Close'], axis=1)
data_set = data.iloc[:, 0:(inputnumber + 3)]

#display(data_set)
data_set.tail(20)

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['^VIX']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2000-01-01 -> 2025-03-24)')


ValueError: If using all scalar values, you must pass an index

In [ ]:
#HERE WE ADD RECENT DATA (backcandles = 30 means 30 days of recent data)
#pd.set_option('display.max_columns', None)
#new_row = {'Open': 0,
#        'High': 0,
#        'Low': 0,
#        'Close': 0,
#        'Adj Close': 0,
#        'Volume': 0,
#        'DMP_16': 0,
#        'DMN_16': 0,
#        'MACD': 0,
#        'ATR': 0,
#        'EMAF': 0,
#        'EMAM': 0,
#        'EMAS': 0}

#data_set = pd.concat([data_set, pd.DataFrame([new_row])], ignore_index=True)
#display(data_set)
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
rows = data_set_scaled[:10].copy()

#print(rows)
X = []
backcandles = 30
for j in range(inputnumber):
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]):
        X[j].append(data_set_scaled[i-backcandles:i, j])
X=np.moveaxis(X, [0], [2])
#display(X[-15:])
X, yi =np.array(X), np.array(data_set_scaled[backcandles:, -1])

#display(X[-15:])


#display(yi[-15:])


y = np.reshape(yi, (len(yi), 1))


#display(y[-15:])

# Calculate splitlimit
splitlimit = int(len(X) * 0.8)
#print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
#print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)
#print(y_train)
np.random.seed(10)

#HERE WE TRAIN AND RUN THE LSTM
for i in range (0, iterations):


    lstm_input = Input(shape=(backcandles, inputnumber), name='lstm_input')
    inputs = LSTM(150, name='first_layer')(lstm_input)
    inputs = Dense(1, name='dense_layer')(inputs)
    output = Activation('linear', name='output')(inputs)
    model = Model(inputs=lstm_input, outputs=output)
    adam = optimizers.Adam()
    model.compile(optimizer=adam, loss='mse')
    model.fit(x=X_train, y=y_train, batch_size=15, epochs=30, shuffle=True, validation_split = 0.1)

#y_test = data_set['Adj Close'][splitlimit:].tolist()
#y_test = y_test[backcandles:]
#y_pred = model.predict(X_test)
#y_test = [[value] for value in y_test]
#y_test = sc.fit_transform(y_test)
#y_test = [value[0] for value in y_test]




    y_pred = model.predict(X_test)
    display(len(y_test))
    display(len(y_pred))


    for i in range(10):
        print(y_pred[i], y_test[i])
    plt.figure(figsize=(16,8))
    plt.plot(y_test, color = 'black', label = 'Test')
    plt.plot(y_pred, color = 'green', label = 'pred')
    plt.legend()
    plt.show()
    #display(y_test[-100:])
    y_test = np.array(y_test).flatten()
    y_pred = np.array(y_pred).flatten()
    df = pd.DataFrame({'test': y_test, 'pred': y_pred})
    #display(df)
    df['testchange'] = (df['test'] > df['test'].shift()).astype(int)
#df['testchange'] = df['test']
    df['predchange'] = (df['pred'] > df['pred'].shift()).astype(int)
#df['predchange'] = (df['pred'] > 0.5).astype(int)
    df['results'] = (df['testchange'] == df['predchange']).astype(int)
#df['results'] = (df['testchange'] == df['pred']).astype(int)
    resultslist = df['results'].tolist()
    #display(df)
    count = resultslist.count(1)
    display(count/len(resultslist))
    testingdata.append(count/len(resultslist))
    pd.set_option('display.max_rows', 200)
    last_10_rows = df.tail(100)
#display(last_10_rows)

In [ ]:
#HERE ARE THE RESULTS
count_ones = df['predchange'].value_counts().get(1, 0)
count_zeros = df['predchange'].value_counts().get(0, 0)
print("Number of 1s:", count_ones)
print("Number of 0s:", count_zeros)
#1s indicate next day positive open - close, while 0s indicate negative open - close

In [ ]:
display(testingdata)

In [ ]:
display(np.round(np.mean(testingdata), decimals=4))
display(np.round(np.std(testingdata), decimals=6))

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print(np.round((elapsed_time / 60), decimals=2))